In [ ]:
import pandas as pd
from generator import Constraint
from utils import *

# Ensure we're running in the right directory
chdir_this_file()

FIELD_SIZE = 3
MIN_CELL_SIZE = 1
MAX_CELL_SIZE = None
CONSTRAINTS = lambda countries: [
    # Some categories are pretty boring to appear multiple times
    Constraint.category_at_most("capital_ending_letter", 1),
    Constraint.category_at_most("capital_starting_letter", 1),
    Constraint.category_at_most("ending_letter", 1),
    # Limit the number of cells a country can appear in
    *Constraint.solutions_at_most(countries.iso.tolist(), 3)
]
CATEGORY_PROBS = {
    "continent": 4,
    "starting_letter": 3,
    "ending_letter": 1.5,
    "capital_starting_letter": 2,
    "capital_ending_letter": .5,
    "flag_colors": 3,
    "landlocked": 2,
    "island": 2,
    "top_20_population": 2.5,
    "bottom_20_population": 2,
    "top_20_area": 2.5,
    "bottom_20_area": 2,
    "elevation_sup5k": 2.5,
    "elevation_sub1k": 2,
    "capital_not_largest": 2,
}

In [ ]:
from preprocessing import Preprocessor
from difficulty import DifficultyEstimator, DifficultyLevel

LANGUAGES = ["de", "en"]
NUM_GAMES = 5000
RUN_NAME = "enhanced-difficulty"

class GenerationPipeline:
    def __init__(self, languge):
        self.language = language
        self.countries = pd.read_json(f"../../public/data/countries/countries-{self.language.lower()}.json", encoding="utf8")
        self.preprocessor = Preprocessor(countries=self.countries,
                                         language=self.language,
                                         field_size=FIELD_SIZE,
                                         min_cell_size=MIN_CELL_SIZE,
                                         max_cell_size=MAX_CELL_SIZE)
        self.constraints = CONSTRAINTS(self.countries)
        self.generator = self.preprocessor.get_generator(self.constraints, CATEGORY_PROBS,
                                                         seed=None, selection_mode="shuffle_setkeys",
                                                         uniform=False, shuffle=True)
        self.estimator = DifficultyEstimator(self.preprocessor)
        
    def generate(self):
        # Generate games
        self.games = list(self.generator.sample_games(n=NUM_GAMES))
        # Difficulty computation
        self.difficulty_info = self.estimator.compute_game_difficulties(self.games)
        # Save games and categories to JSON file
        # self.preprocessor.save_games(self.games, name=RUN_NAME)

pipelines = {}
for language in LANGUAGES:
    pipelines[language] = GenerationPipeline(language)
    pipelines[language].generate()

In [ ]:
EN = pipelines["en"]
DE = pipelines["de"]
EN.games[0].data

In [ ]:
from importlib import reload
import difficulty as diff
reload(diff)
EN.estimator = diff.DifficultyEstimator(EN.preprocessor)
EN.estimator = diff.DifficultyEstimator(EN.preprocessor)
EN.difficulty_info = EN.estimator.compute_game_difficulties(EN.games)
DE.difficulty_info = DE.estimator.compute_game_difficulties(DE.games)

In [ ]:
EN.estimator.cell_info.sort_values("difficulty")

In [ ]:
EN.estimator.cell_info["level"].value_counts()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# x = "solution_difficulty"
# y = "min_solution_difficulty"
# plt.scatter(EN.estimator.cell_info[x], EN.estimator.cell_info[y])
# plt.xlabel(x)
# plt.ylabel(y)
# plt.show()

sns.pairplot(EN.estimator.cell_info, vars=["difficulty", "solution_difficulty", "min_solution_difficulty"])
plt.show()

In [ ]:
EN.difficulty_info

In [ ]:
[lvl for lvl in DifficultyLevel]

In [ ]:
EN.difficulty_info

In [ ]:
EN.difficulty_info["cell_levels"].apply(lambda lvls: ", ".join(sorted(list(lvls.keys()))))

In [ ]:
def count_cell_levels(preprocessor, cell_indices):
    levels = [str(EN.estimator.cell_info.loc[i, "level"]) for i in cell_indices]
    return {str(lvl): levels.count(str(lvl)) for lvl in DifficultyLevel}
EN.difficulty_info["cell_levels"] = EN.difficulty_info["cell_indices"].apply(lambda ii: count_cell_levels(EN, ii))
EN.difficulty_info["cell_levels_str"] = EN.difficulty_info["cell_levels"].apply(lambda lvls: f'{lvls["easy"]}{lvls["medium"]}{lvls["hard"]}')
# EN.difficulty_info.sort_values("cell_levels_str", ascending=False)
cell_levels_str_counts = EN.difficulty_info["cell_levels_str"].value_counts()

In [ ]:
cell_levels_str_counts

In [ ]:
stats = pd.DataFrame([
    {
        "easy": easy,
        "medium": 9 - easy - hard,
        "hard": hard,
        "num": int(cell_levels_str_counts.get(f"{easy}{9 - easy - hard}{hard}", 0)),
    }
    for easy in range(10)
    for hard in range(10 - easy)
])
plt.scatter(
    stats["easy"],
    stats["hard"],
    s=stats["num"],
)
plt.xlabel("easy")
plt.ylabel("hard")
plt.show()
plt.legend()

In [ ]:
game = EN.games[0]

In [ ]:
game.rows, game.cols

In [ ]:
(row_cat, row_val), (col_cat, col_val) = game.rows[0], game.cols[0]
row_cat, col_cat

In [ ]:
row_cat.sets

In [ ]:
EN.estimator.category_info

In [ ]:
categories["flag_colors"].sets

In [ ]:
import numpy as np
from category import Category


def category_entropy(category: Category, preprocessor: Preprocessor):
    contained = set().union(*category.sets)  # type: ignore
    all_codes = set(preprocessor.df["iso"])
    rest = all_codes - contained
    sizes = np.array([len(s) for s in category.sets] + ([len(rest)] if rest else []))  # type: ignore
    print(category.key, sizes)
    # if sum(sizes) < len(category.values):
    ps = 1 / sizes
    return -1 * (ps * np.log(ps)).sum()

categories = EN.preprocessor.categories
pd.DataFrame(
    [
        {
            "key": cat.key,
            "entropy": category_entropy(cat, EN.preprocessor),
        }
        for cat in categories.values()
    ]
).sort_values("entropy")

In [ ]:
EN.estimator.cell_info.apply(
    lambda row: (
        len(EN.preprocessor.categories[row["row_cat"]].sets[row["row_val"]]),
        len(EN.preprocessor.categories[row["col_cat"]].sets[row["col_val"]])
    )
, axis=1)

In [ ]:
EN.preprocessor.cells

In [ ]:
EN.estimator.df.groupby(EN.estimator.df["difficulty"].apply(int)).agg({"iso": list, "difficulty": "count"})

In [ ]:
EN.estimator.df.sort_values("difficulty").tail(10)

In [ ]:
EN.estimator.cell_info[
    (EN.estimator.cell_info["difficulty"] > 4.5)
    & (EN.estimator.cell_info["difficulty"] < 5.5)
]

In [ ]:
EN.estimator.cell_info.sort_values("difficulty").iloc[1900:2000]

In [ ]:
EN.difficulty_info.sort_values("difficulty").head(50)

## From `game_stats.ipynb`

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
plt.hist([game.sample_tries for game in EN.games], label="EN", bins=50, rwidth=.9, alpha=.75)
plt.hist([game.sample_tries for game in DE.games], label="DE", bins=50, rwidth=.9, alpha=.75)
plt.title("Number of tries for game generation")
plt.legend()
plt.show()

In [ ]:
# sample_tries = pd.Series([game.sample_tries for game in games]).value_counts()
# sample_tries = np.array([sample_tries.get(i, 0) for i in range(100)])
# p = (1 - sample_tries / np.roll(np.where(sample_tries != 0, sample_tries, np.nan), 1))[1:]
# plt.scatter(x=p, y=np.zeros_like(p))
# plt.show()

In [ ]:
DE.difficulty_info["max_occurences"] = DE.difficulty_info["game"].apply(
    lambda game: pd.Series(sum(sum(game.solutions, []), [])).value_counts().max()
)
EN.difficulty_info["max_occurences"] = EN.difficulty_info["game"].apply(
    lambda game: pd.Series(sum(sum(game.solutions, []), [])).value_counts().max()
)
DE.difficulty_info["max_occurences"].describe()

In [ ]:
plt.hist(EN.difficulty_info["max_occurences"], bins=range(10), rwidth=.9)
plt.title("Maximum number of cells a country appears in")
plt.show()

In [ ]:
difficulty_info

In [ ]:
plt.hist(difficulty_info["max_cell_difficulty"], bins=20, rwidth=.9)
plt.title("Max cell difficulty")
plt.show()

In [ ]:
for game in difficulty_info.nlargest(20, "max_cell_difficulty")["game"]:
    print(game.data)
    display(game.to_dataframe(solution=True))

In [ ]:
scatter = plt.scatter(x=difficulty_info["avg_cell_difficulty"], y=difficulty_info["max_cell_difficulty"], c=difficulty_info["level"])
plt.xlabel("Average cell difficulty")
plt.ylabel("Maximum cell difficulty")
plt.title("Distribution of game difficulty")
plt.xlim([0,10])
plt.ylim([0,10])
ax = plt.gca()
legend = ax.legend(scatter.legend_elements()[0], ["Easy", "Medium", "Hard"], loc="lower right", title="Difficulty Level")
ax.add_artist(legend)
plt.show()

print(difficulty_info["level"].value_counts())

In [ ]:
cell_info = estimator.cell_info
cell_info.nsmallest(20, "difficulty")

In [ ]:
plt.hist(cell_info["solution_difficulty"], bins=20, rwidth=.9)
plt.title("Cell solution difficulty")
plt.show()

In [ ]:
plt.scatter(x=cell_info["row_col_difficulty"], y=cell_info["solution_difficulty"])
plt.xlabel("Row-col difficulty")
plt.ylabel("Solution difficulty")
plt.show()

plt.scatter(x=cell_info["row_difficulty"], y=cell_info["col_difficulty"], c=cell_info["solution_difficulty"])
plt.xlabel("Row difficulty")
plt.ylabel("Col difficulty")
plt.show()
# plt.scatter(x=cell_info["row_col_difficulty"], y=cell_info["row_col_difficulty_harmonic"])
# plt.show()

In [ ]:
# difficulty_info.nlargest(10, "difficulty")

In [ ]:


# plt.scatter(x=np.log(df["gdp"]) - np.log(df["population"]), y=df["gdp_per_capita"])
# plt.scatter(x=np.log(df["population"]), y=np.log(df["gdp"]), c=df["difficulty"])
# plt.scatter(x=np.log(df["population"]), y=np.log(df["gdp"]), c=df["difficulty"])
# plt.show()